In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras.applications.vgg19 import VGG19
import numpy as np
import pandas as pd
import PIL.Image as Image
import os
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications import xception
from keras.applications.densenet import DenseNet121
from keras.applications.inception_v3 import InceptionV3

In [5]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Colab Notebooks/Breast Cancer/BUID Dataset.zip', 'r') as zipObj:

     zipObj.extractall('BreastCancer')

In [6]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import splitfolders
input_folder = '/content/BreastCancer/BUID/'

In [8]:
splitfolders.ratio(input_folder, output="dataset3",
                   seed=123, ratio=(.8, .1, .1), 
                   group_prefix=None)

Copying files: 780 files [00:00, 1544.18 files/s]


In [9]:
BATCH_SIZE = 64
IMAGE_SIZE = 224
CHANNELS=3
EPOCHS = 50

In [10]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/train',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/test',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
'/content/dataset3/val',
image_size= (IMAGE_SIZE, IMAGE_SIZE), batch_size= BATCH_SIZE)

Found 623 files belonging to 3 classes.
Found 80 files belonging to 3 classes.
Found 77 files belonging to 3 classes.


In [11]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/dataset3/train')

class_names= train_dataset.class_names
class_names

Found 623 files belonging to 3 classes.


['benign', 'malignant', 'normal']

In [12]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [13]:
data_augmentation = tf.keras.Sequential([
layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
layers.experimental.preprocessing.RandomRotation(0.2),
])

In [14]:
training_ds = train_ds.map(
lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [15]:
InceptionV31 = keras.applications.InceptionV3(input_shape=(224,224,3),include_top=False,weights='imagenet')
InceptionV31.output

87924736/87910968 [==============================] - 0s 0us/step


<KerasTensor: shape=(None, 5, 5, 2048) dtype=float32 (created by layer 'mixed10')>

In [16]:
for layer in InceptionV31.layers:
  InceptionV31.trainable = False

In [17]:
x= Flatten()(InceptionV31.output)
prediction=Dense(5,activation='softmax')(x)
model=Model(inputs=InceptionV31.input,outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [18]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [19]:
import time
start = time.time()
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)
print("Total time: ", time.time() - start, "seconds")

Epoch 1/50
10/10 [==============================] - 28s 587ms/step - loss: 149.2529 - accuracy: 0.4559 - val_loss: 119.7045 - val_accuracy: 0.3247
Epoch 2/50
10/10 [==============================] - 5s 211ms/step - loss: 68.8848 - accuracy: 0.5778 - val_loss: 31.7973 - val_accuracy: 0.6623
Epoch 3/50
10/10 [==============================] - 5s 211ms/step - loss: 42.1031 - accuracy: 0.6790 - val_loss: 33.6218 - val_accuracy: 0.6883
Epoch 4/50
10/10 [==============================] - 5s 212ms/step - loss: 23.1428 - accuracy: 0.7239 - val_loss: 34.0531 - val_accuracy: 0.6364
Epoch 5/50
10/10 [==============================] - 5s 220ms/step - loss: 10.9875 - accuracy: 0.7994 - val_loss: 24.8179 - val_accuracy: 0.6623
Epoch 6/50
10/10 [==============================] - 5s 214ms/step - loss: 10.5357 - accuracy: 0.7929 - val_loss: 35.1107 - val_accuracy: 0.5065
Epoch 7/50
10/10 [==============================] - 5s 213ms/step - loss: 6.5126 - accuracy: 0.8250 - val_loss: 15.1617 - val_accurac

In [20]:
scores = model.evaluate(test_ds)

2/2 [==============================] - 1s 946ms/step - loss: 43.1822 - accuracy: 0.6875
